Copyright 2020 Montvieux Ltd

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import requests
from io import BytesIO
import PIL.Image
from IPython.display import display,clear_output,HTML
from IPython.display import Image as DisplayImage
import base64
import json
from io import StringIO
import ipywidgets as widgets
import sys
import time
import imageio
import numpy as np
import io

from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.evaluation import evaluate_policy

from plark_game import classes
from gym_plark.envs import plark_env,plark_env_guided_reward,plark_env_top_left,plark_env_sonobuoy_deployment, panther_env_reach_top


from stable_baselines3 import DQN, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

import helper 
import datetime
import os

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


%matplotlib inline
%load_ext autoreload
%autoreload 2


2023-05-03 16:17:39.072918: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 16:17:40.106368: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [4]:
env = panther_env_reach_top.PantherEnvReachTop(config_file_path='/Components/plark-game/plark_game/game_config/10x10/balanced.json')
check_env(env)

INFO:gym_plark.envs.plark_env:plark.kwargs :{'driving_agent': 'panther', 'render_height': 250, 'render_width': 310}
INFO:gym_plark.envs.plark_env:self.image_based :True
INFO:gym_plark.envs.plark_env:config filepath: /Components/plark-game/plark_game/game_config/10x10/balanced.json
INFO:plark_game.classes.environment:Opening config from:/Components/plark-game/plark_game/game_config/10x10/balanced.json


ValueError: No configuration file provided

## Training loop

In [ ]:

n_eval_episodes = 10
training_steps = 200

#env.driving_agent = 'panther'

model = PPO('CnnPolicy', env)
#model = PPO('CnnPolicy', 'plark-v0') 
#model = A2C('CnnPolicy', env)
model.learn(training_steps)

print("****** STARTING EVALUATION *******")
mean_reward, n_steps = evaluate_policy(model, env, n_eval_episodes=n_eval_episodes, deterministic=False, render=False, callback=None, reward_threshold=None, return_episode_rewards=False)
print("****** EVALUATION FINISHED *******")
print("Mean Reward is " + str(mean_reward))
print("Number of steps is " + str(n_steps))

In [ ]:
#Save model 
basicdate = str(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

basepath = '/data/agents/models'
exp_name = 'test_' + basicdate
exp_path = os.path.join(basepath, exp_name)

print(exp_path)


In [ ]:



modeltype = 'PPO'
modelplayer = env.driving_agent 
render_height = env.render_height
render_width = env.render_width
image_based = True
helper.save_model(exp_path,model,modeltype,modelplayer,render_height,render_width,image_based,basicdate)

# making the video

In [ ]:
video_path = '/test.mp4'
basewidth,hsize = helper.make_video(model,env,video_path)

In [ ]:
video = io.open(video_path, 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" width="'''+str(basewidth)+'''" height="'''+str(hsize)+'''" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii')))